# メモ書き
## MIXED PRECISION TRAINING
- 最近のDNNモデルは徐々に巨大化しており、それに伴いより多くの計算やメモリーへのリソースが学習の際に必要となる。それらの必要となるリソースは、精度を低くすることにより、減らすことができる。
- 学習や推論等を行う速度は以下の3つによって決まる。
    1. 演算速度
    2. メモリ速度
    3. レイテンシ<br>
    最初の2つは扱う精度を低くすることにより、向上させることができる。<br>
        - 精度を低くすることにより(例:FP32→FP16)、同じ値をより少ないビット数でメモリに持たせることができる。
        - より少ない精度を使用することにより、演算しなければならないビット数も減らせることができる。例を挙げると、FP16を使用して最新のGPUで演算を行うと、FP32で演算した時と比べて、2倍～8倍のスループットを実現する。
- 今回提案する手法は、FP32の代わりにFP16を使用し、様々なNNモデルでaccuracyを保ちつつ、演算速度をあげることを検証する。
- FP32からFP16に代えても、モデルのaccuracyを保つために以下3つの提案する。
    1. FP32で重みの値を保持する
    2. 勾配が0になることを防ぐためのロススケーリング
    3. FP16とFP32の混合演算
    <br><span style="color: red; ">**※**</span>なお上3つのことを行う上で、モデルやハイパーパラメータを変更する必要はない
***
## Training With Mixed Precision :: DEEP LEARNING SDK DOCUMENTATION
- Mixed Presicion Trainingを使用するためには、以下2つのステップが必要。<br>
    1. 計算上FP16で事が足りる箇所はFP16を適用する<br>
    2. 勾配の値が小さくなっても保持するためにロススケーリングを使用する<br>
- より小さな精度で演算を行うメリットは以下の2つ<br>
    1. **メモリー使用量を削減**<br>
    例を挙げると、普段使用する精度はFP32(32ビット必要)。対してFP16(16ビット必要)。すなわちFP32からFP16に変更することで16ビッド(32ビット-16ビット)も削減できる。そうすることにより、より大きなモデル、またはより大きなミニバッチで学習させることができる。<br>
    2. **学習時間、または推論時間の短縮**<br>
    上の例で挙げたように、FP16を使用することにより、FP32を使用する時と比べて、アクセスするバイト数を半分(16ビット)にできる。そのため、計算時間も短縮できる。NVIDIAのGPUでは演算にFP16を使用すると、FP32で計算する時と比べて、最大8倍のスループットを実現する。